# Derivation and Abuse of the EKF Method for Real-World Slammage
Kalman Filters are used for recursively estimating the state of a linear system with noise or uncertainty. Extended Kalman Filters (EKF) allow this technique to be applied to nonlinear systems. This facilitates an EKF-based solution to the SLAM problem.


### Part 0 - Preface
This guide page assumes a relative degree of comfort with probability and linear algebra. None of the math here is super nasty but there are a lot of steps so being comfortable with matrix operations (i.e. multiplications, what is a Transposition and why is it used, what is a Jacobian and why is it used), and clearly understanding what an expectation of a distribution is will go a long way. I will briefly summarize these now for the precocious uninitiated:

1. WTF is a Transposition and why is it used?: $\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}, \quad \mathbf{x}^T = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix}$ Now we can multiply these two vectors together. We can also do the same thing with matrices.

2. WTF is a Jacobian and why is it used?: Special matrix we use to get the the first-order derivative of a vectorial function: $J_f = \begin{bmatrix}
\frac{\partial f_1(x)}{\partial x_1} & \frac{\partial f_1(x)}{\partial x_2} & \cdots & \frac{\partial f_1(x)}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial f_n(x)}{\partial x_1} & \frac{\partial f_n(x)}{\partial x_2} & \cdots & \frac{\partial f_n(x)}{\partial x_n}
\end{bmatrix}, \quad J_ff(\cdot) = \nabla f(\cdot)$

3. WTF is an expectation and why is it used?: Most likely value of a Random Variable. Think of it as the weighted average across all outcomes, with each outcome multiplied by it's respective probability: $$\mathbb{E}[X] = \int_{-\infty}^{\infty} x \, p(x) \, dx$$ or $$\mathbb{E}[X] = \sum_{x} x \, p(x)$$ depending on if the RV is continuous or discrete.

I also want to make the disclaimer that while robotics is used as the context for explaining why some of the math below works out the way it does, it's not the only way. EKF can be used for other things as well, but this whole piece is centered around explaining it in the context of SLAM, and so robotics becomes the chosen medium.
### Part 1 - The Derivation

#### 1.0 - Preliminaries

For now we will just focus on the robot's state, $x_k$. We will define the state vector distribution as some function of the previous state, $f(x_{k-1})$, with zero-mean gaussian noise, $w_{k-1}$, added to it to account for uncertainty in the previous state. We will define our observation vector distribution, $z_k$, as some function of the current state, $f(x_k)$, with zero-mean gaussian noise, $v_{k}$, added to it to account for observation uncertainty (sensor noise). This manifests the following two equations:


>$$
>(1) \quad x_k = f(x_{k-1}) + w_{k-1} 
>$$
>$$(2) \quad z_k = f(x_k) + v_{k-1} 
>$$
